In [1]:
### TEST
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

# 프로젝트 루트: notebooks/Jang -> notebooks -> project_root
project_root = nb_dir.parents[1]  # /data/ephemeral/pro-nlp-generationfornlp-nlp-13

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [2]:
from transformers import AutoTokenizer

from src.prompt.prompt_registry import PromptRegistry
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.data.data_loader import DataConfig, make_train_valid_dataset
from src.data.tokenizer_wrapper import TokenizerConfig

In [3]:
policy = {
    "system": {4: "v1", 5: "v1"},
    "user":   {4: "v1", 5: "v1"},
}

prompt_cfg = PromptConfig(
    policy=policy,
    mode="train",
    verbose=True
)

data_cfg = DataConfig(
    train_path=project_root / "data" / "train.csv",
    valid_ratio=0.1,
    seed=42,
    do_split=True,
)

tokenize_cfg_train = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=False,
)

tokenize_cfg_gen = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=True,
)

MODEL_NAME = "Qwen/Qwen3-14B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

In [6]:
ds = make_train_valid_dataset(
    data_cfg=data_cfg,
    prompt_cfg=prompt_cfg,
    tokenize_cfg_train=tokenize_cfg_train,
    tokenize_cfg_gen=tokenize_cfg_gen,
    tokenizer=tokenizer,
)

template loading 완료: system=2, user_4=2, user_5=2
template loading 완료: system=2, user_4=2, user_5=2


Build train messages:   0%|          | 0/1827 [00:00<?, ? examples/s]

Serialize train to text:   0%|          | 0/1827 [00:00<?, ? examples/s]

Build valid messages (teacher forcing):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid to text:   0%|          | 0/204 [00:00<?, ? examples/s]

Build valid_gen messages (prompt only):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid_gen to text (+meta):   0%|          | 0/204 [00:00<?, ? examples/s]

In [8]:
ds['train'][0]['text']

'<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>\n<|im_start|>user\n### 지문\n“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했다.한편 Fed의 양적완화 축소(테이퍼링)가 신흥국에 미친 영향도 크지 않았던 것으로 나타났다. 테이

In [9]:
ds['train'][0]

{'id': 'generation-for-nlp-2661',
 'label': 3,
 'text': '<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>\n<|im_start|>user\n### 지문\n“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했

In [10]:
from transformers import AutoTokenizer
from trl import DataCollatorForCompletionOnlyLM

response_template = "<|im_start|>assistant\n" 

data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)


sample = ds['train'][0]
print(f"원본 텍스트 일부:\n{sample['text']}...")

tokenized_sample = tokenizer(sample['text'], add_special_tokens=False)
batch = data_collator([tokenized_sample])

input_ids = batch["input_ids"][0]
labels = batch["labels"][0]

print("✅ 처리 성공!")
print(f"Input IDs 길이: {len(input_ids)}")
print(f"Labels 길이: {len(labels)}")

원본 텍스트 일부:
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했다.한편 Fed의 양적완화 축소(테이퍼링)가 신흥국에 미친 영향도 크지 않았던 것으로 나타났다

In [11]:
labels[-7:]

tensor([151667,    271, 151668,    271,     18, 151645,    198])

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,
)

prompt = "안녕하세요. 너의 소개를 1줄로 해줘."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

안녕하세요. 너의 소개를 1줄로 해줘. 그리고 1000자 이내로 간단한 자기소개를 해줘. 

안녕하세요! 저는 Qwen, 알리바바 그룹이 개발한 초대규모 언어 모델입니다. 저는 다양한 주제에 대해 대화하고, 질문에 답하고, 창의적인 글을 작성하며, 다양한 언어를 지원하는 등 다양한 기능을 제공합니다. 저는 지속적으로 학습과 업데이트


In [18]:
prompt = "안녕하세요. 너의 소개를 1줄로 해줘."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

안녕하세요. 너의 소개를 1줄로 해줘. 그리고 1000자 이내로 간단한 자기소개를 해줘. 

안녕하세요! 저는 Qwen, 알리바바 그룹이 개발한 초대규모 언어 모델입니다. 저는 다양한 주제에 대해 대화하고, 질문에 답하고, 창의적인 글을 작성하며, 다양한 언어를 지원하는 등 다양한 기능을 제공합니다. 저는 지속적으로 학습과 업데이트


In [19]:
out1 = outputs

In [21]:
messages = [
    {"role": "system", "content": "너는 한국어로 간단히 답하는 비서야."},
    {"role": "user", "content": "안녕하세요. 너의 소개를 1줄로 해줘."},
]

prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=False,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

gen_ids = outputs[0][inputs["input_ids"].shape[-1]:]
print(tokenizer.decode(gen_ids, skip_special_tokens=True))

안녕하세요, 저는 사용자의 요청에 따라 간단히 대답해주는 비서입니다.


In [ ]:
messages = [
    {"role": "system", "content": "너는 한국어로 간단히 답하는 비서야."},
    {"role": "user", "content": "안녕하세요. 너의 소개를 1줄로 해줘."},
]

prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True,
)

inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=False,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

gen_ids = outputs[0][inputs["input_ids"].shape[-1]:]
print(tokenizer.decode(gen_ids, skip_special_tokens=True))

<think>
Okay, the user asked for a one-line introduction in Korean. Let me think about how to respond.

First, I need to keep it concise. They want a short introduction, so I shouldn't add extra details. My main role is


: 

In [ ]:

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
out2 = outputs